In [ ]:

import pandas as pd

df_train = pd.read_csv('/content/drive/MyDrive/00_signate/中古自動車/train.csv')
df_train.set_index('id', inplace=True)
df_train.head()

,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,price
id,,,,,,,,,,,,,,,
0,nashville,1949,bmw,excellent,6 cylinders,gas,115148,clean,manual,rwd,mid-size,convertible,orange,NaN,27587
1,state college,2013,toyota,fair,8 cylinders,gas,172038,clean,automatic,rwd,full-size,sedan,silver,pa,4724
2,wichita,1998,ford,good,6 cylinders,gas,152492,clean,automatic,fwd,full-size,SUV,silver,ks,10931
3,albany,2014,ford,excellent,4 cylinders,gas,104118,clean,manual,fwd,mid-size,SUV,blue,ny,16553
4,redding,2005,ford,excellent,6 cylinders,gas,144554,clean,manual,fwd,mid-size,sedan,red,ca,5158


In [12]:
!git config --global user.email "nomukapi@outlook.jp"
!git config --global user.name "nomukapi"


In [17]:
%cd /content/drive/MyDrive/00_signate
!git clone https://github.com/nomusanma/test2.git


/content/drive/MyDrive/00_signate
Cloning into 'test2'...


In [18]:
!git init


Reinitialized existing Git repository in /content/drive/MyDrive/00_signate/.git/


In [22]:
!cd LightGBM
!git reset --hard
!cd ..


HEAD is now at db23d7c コミットメッセージ


In [24]:
# test2/ ディレクトリ内の.gitディレクトリを削除
!rm -rf test2/.git

# test2/ ディレクトリを追加
!git add test2/
!git commit -m "test2ディレクトリの追加"


On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
  (commit or discard the untracked or modified content in submodules)
	modified:   LightGBM (untracked content)

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
import pandas as pd

df_test = pd.read_csv('/content/drive/MyDrive/00_signate/中古自動車/test.csv')
df_test.set_index('id', inplace=True)
df_test.head()

,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
id,,,,,,,,,,,,,,
27532,western slope,2015,chevrolet,excellent,4 cylinders,gas,92553,clean,automatic,fwd,full-size,SUV,red,NaN
27533,roseburg,2013,nissan,like new,4 cylinders,gas,134385,salvage,automatic,fwd,mid-size,sedan,black,or
27534,akron / canton,2011,volkswagen,good,4 cylinders,gas,102489,clean,automatic,fwd,full-size,sedan,black,oh
27535,denver,2016,jeep,excellent,6 cylinders,diesel,64310,clean,automatic,4wd,mid-size,SUV,red,co
27536,hickory / lenoir,1999,honda,excellent,8 cylinders,gas,180839,rebuilt,automatic,4wd,mid-size,SUV,silver,nc


In [ ]:
# Drop rows with missing values
# df = df.dropna()
import numpy as np

def process(df):
    df["cylinders"] = df["cylinders"].fillna("none")
    df["fuel"] = df["fuel"].fillna("gas")
    df["title_status"] = df["title_status"].fillna("none")
    df["type"] = df["type"].fillna("none")
    df["state"] = df["state"].fillna("none")
    df["paint_color"] = df["paint_color"].fillna("none")
    # Correct 'year' values
    df['year'] = df['year'].apply(lambda x: int(str(x).replace('29', '19')) if str(x)[:2]=='29' else int(str(x).replace('30', '20')))
    df["year_sabun"] =2023-df["year"]
    df["car"]=df["manufacturer"]+"_"+df["type"]+"_"+df["drive"]

    # df.loc[df['odometer'] <= 0, 'odometer'] = 0



    return df

df_train=process(df_train)
df_train["price_log"]=np.log1p(df_train["price"])
df_test=process(df_test)

# Separate features and target variable
X = df_train.drop(columns=['price',"price_log"])
y = df_train["price_log"]

X_new_test=df_test.copy()

In [ ]:
# カテゴリカル変数のリスト
categorical_features_full =["car",'region', 'manufacturer', 'condition', 'cylinders', 'fuel', 'title_status', 'transmission', 'drive', 'size', 'type', 'paint_color', 'state']
categorical_features =["car",'region', 'manufacturer', 'condition', 'cylinders', 'fuel', 'title_status', 'transmission', 'drive', 'size', 'type', 'paint_color', 'state']
# categorical_features =["car", 'manufacturer', 'condition', 'cylinders', 'fuel', 'title_status', 'transmission', 'drive', 'size', 'type']
add_list=[]
# すべてのカテゴリカル変数のペアの組み合わせを生成
for i, feature1 in enumerate(categorical_features):
    for j, feature2 in enumerate(categorical_features):
        if i < j:  # 同じ特徴の組み合わせを避ける
            new_feature_name = feature1 + "_" + feature2
            X[new_feature_name] = X[feature1].astype(str) + "_" + X[feature2].astype(str)
            X_new_test[new_feature_name] = X_new_test[feature1].astype(str) + "_" + X_new_test[feature2].astype(str)
            add_list.append(new_feature_name)

categorical_features=categorical_features_full+add_list

# 結果の確認
X.head()


,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,...,drive_size,drive_type,drive_paint_color,drive_state,size_type,size_paint_color,size_state,type_paint_color,type_state,paint_color_state
id,,,,,,,,,,,,,,,,,,,,,
0,nashville,1949,bmw,excellent,6 cylinders,gas,115148,clean,manual,rwd,...,rwd_mid-size,rwd_convertible,rwd_orange,rwd_none,mid-size_convertible,mid-size_orange,mid-size_none,convertible_orange,convertible_none,orange_none
1,state college,2013,toyota,fair,8 cylinders,gas,172038,clean,automatic,rwd,...,rwd_full-size,rwd_sedan,rwd_silver,rwd_pa,full-size_sedan,full-size_silver,full-size_pa,sedan_silver,sedan_pa,silver_pa
2,wichita,1998,ford,good,6 cylinders,gas,152492,clean,automatic,fwd,...,fwd_full-size,fwd_SUV,fwd_silver,fwd_ks,full-size_SUV,full-size_silver,full-size_ks,SUV_silver,SUV_ks,silver_ks
3,albany,2014,ford,excellent,4 cylinders,gas,104118,clean,manual,fwd,...,fwd_mid-size,fwd_SUV,fwd_blue,fwd_ny,mid-size_SUV,mid-size_blue,mid-size_ny,SUV_blue,SUV_ny,blue_ny
4,redding,2005,ford,excellent,6 cylinders,gas,144554,clean,manual,fwd,...,fwd_mid-size,fwd_sedan,fwd_red,fwd_ca,mid-size_sedan,mid-size_red,mid-size_ca,sedan_red,sedan_ca,red_ca


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize a dictionary to store label encoders for each categorical variable
label_encoders = {}
categorical_vars=categorical_features
for var in categorical_vars:
    le = LabelEncoder()

    # Fit the encoder on the combined data to ensure all categories are captured
    combined_data = pd.concat([X[var], X_new_test[var]], axis=0).astype(str)
    le.fit(combined_data)

    # Transform the training and test data
    X[var] = le.transform(X[var].astype(str))
    X_new_test[var] = le.transform(X_new_test[var].astype(str))

    # Store the encoder for potential use later
    label_encoders[var] = le

X.head()


,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,...,drive_size,drive_type,drive_paint_color,drive_state,size_type,size_paint_color,size_state,type_paint_color,type_state,paint_color_state
id,,,,,,,,,,,,,,,,,,,,,
0,215,1949,39,0,5,2,115148,0,1,2,...,21,30,30,137,46,46,191,25,100,329
1,321,2013,72,1,6,2,172038,0,0,2,...,18,38,33,143,24,21,91,117,488,457
2,358,1998,46,2,5,2,152492,0,0,1,...,9,14,21,68,14,21,68,9,16,434
3,3,2014,46,0,3,2,104118,0,1,1,...,12,14,13,87,44,41,193,1,35,87
4,264,2005,46,0,5,2,144554,0,1,1,...,12,24,20,56,54,48,162,116,453,370


In [ ]:
X_new_test

,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,...,drive_size,drive_type,drive_paint_color,drive_state,size_type,size_paint_color,size_state,type_paint_color,type_state,paint_color_state
id,,,,,,,,,,,,,,,,,,,,,
27532,357,2015,42,0,3,2,92553,0,0,1,...,9,14,20,85,14,20,85,8,33,399
27533,271,2013,63,3,3,2,134385,6,0,1,...,12,24,12,90,54,40,196,108,487,38
27534,2,2011,74,2,3,2,102489,0,0,1,...,9,24,12,88,24,12,88,108,485,36
27535,74,2016,52,0,5,0,64310,0,0,0,...,4,0,8,5,44,48,163,8,5,371
27536,134,1999,48,0,6,2,180839,5,0,0,...,4,0,9,27,44,49,185,9,27,445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55064,121,2016,47,0,5,2,90902,5,0,1,...,9,25,12,78,25,12,78,120,527,26
55065,174,2012,63,0,3,2,27234,5,0,1,...,12,24,22,86,54,50,192,118,483,504
55066,251,2002,39,0,5,2,99761,0,0,2,...,18,31,25,107,17,13,55,32,120,55


In [ ]:
# from sklearn.model_selection import train_test_split
# import xgboost as xgb
# from sklearn.linear_model import LinearRegression, Ridge
# from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
# from sklearn.svm import SVR
# from sklearn.neural_network import MLPRegressor
# from sklearn.model_selection import KFold
# from sklearn.preprocessing import MinMaxScaler
# import numpy as np
# import pandas as pd

# targets_for_encoding = ['price']
# # , 'odometer', 'year'
# def target_encode_with_multiple_targets(X,y):
#     # Copy the original data for target encoding
#     X_encoded = X.copy()
#     X_encoded['price'] = y

#     # Apply target encoding and create new features for each categorical variable and target
#     for target in targets_for_encoding:
#         for var in categorical_vars:
#             # Calculate statistics for target encoding
#             target_mean = X_encoded.groupby(var)[target].mean()
#             target_min = X_encoded.groupby(var)[target].min()
#             target_max = X_encoded.groupby(var)[target].max()

#             # Create new features with the calculated statistics
#             X_encoded[var + f'_{target}_mean_enc'] = X_encoded[var].map(target_mean)
#             # X_encoded[var + f'_{target}_min_enc'] = X_encoded[var].map(target_min)
#             X_encoded[var + f'_{target}_max_enc'] = X_encoded[var].map(target_max)
#     X_encoded.drop('price', axis=1, inplace=True)

#     return X_encoded

# def target_encode_evaluation_data_with_multiple_targets(X_eval, X_train, y_train):
#     # Copy the evaluation data for target encoding
#     X_encoded_eval = X_eval.copy()

#     # Add 'price' to training data for target encoding calculations
#     X_train_encoded = X_train.copy()
#     X_train_encoded['price'] = y_train

#     # Apply target encoding using statistics from the TRAINING data
#     for target in targets_for_encoding:
#         for var in categorical_vars:
#             # Calculate statistics for target encoding using TRAINING data
#             target_mean = X_train_encoded.groupby(var)[target].mean()
#             target_min = X_train_encoded.groupby(var)[target].min()
#             target_max = X_train_encoded.groupby(var)[target].max()

#             # Apply the calculated statistics to the EVALUATION data
#             X_encoded_eval[var + f'_{target}_mean_enc'] = X_encoded_eval[var].map(target_mean)
#             # X_encoded_eval[var + f'_{target}_min_enc'] = X_encoded_eval[var].map(target_min)
#             X_encoded_eval[var + f'_{target}_max_enc'] = X_encoded_eval[var].map(target_max)

#     # Handle any NaN values which might occur if there's a new category in evaluation data
#     X_encoded_eval.fillna(X_train_encoded.mean(), inplace=True)

#     return X_encoded_eval

# # Apply target encoding to the full dataset
# #X_encoded = target_encode_full_data(X, y)
# X_train_scaled =target_encode_with_multiple_targets(X,y)
# X_test_scaled=target_encode_evaluation_data_with_multiple_targets(X_new_test,X,y)
# # Split the encoded data into training and test sets (30% of the data will be used for testing)
# # X_train_encoded, X_test_encoded, y_train, y_test = train_test_split(X_encoded, y, test_size=0.3, random_state=42)

# # Select only numeric columns for scaling
# # X_train_encoded_numeric = X_train_encoded.select_dtypes(include=[np.number])
# # X_test_encoded_numeric = X_test_encoded.select_dtypes(include=[np.number])

# # Scaling
# # scaler = MinMaxScaler()
# # X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_encoded_numeric), columns=X_train_encoded_numeric.columns)
# # X_test_scaled = pd.DataFrame(scaler.transform(X_test_encoded_numeric), columns=X_test_encoded_numeric.columns)
# # X_train_scaled=X_train_encoded_numeric
# # X_test_scaled=X_test_encoded_numeric
# X_train_scaled, X_traintest_scaled, y_train, y_test = train_test_split(X_train_scaled, y, test_size=0.3, random_state=42)

In [ ]:
X_all= pd.concat([X, X_new_test], axis=0)

In [ ]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

# 1. 'price' のターゲットエンコーディング用に変数を設定
targets_for_price = ['price']
targets_for_encoding = ['price']
# 2. 'all' からの他のターゲットエンコーディング用に変数を設定
targets_for_all = ['odometer', 'year']

def target_encode_with_specified_targets(X, y, targets):
    X_encoded = X.copy()
    X_encoded['price'] = y

    # Apply target encoding and create new features for each categorical variable and target
    for target in targets:
        for var in categorical_vars:
            # Calculate statistics for target encoding
            target_mean = X_encoded.groupby(var)[target].mean()
            target_min = X_encoded.groupby(var)[target].min()
            target_max = X_encoded.groupby(var)[target].max()

            # Create new features with the calculated statistics
            X_encoded[var + f'_{target}_mean_enc'] = X_encoded[var].map(target_mean)
            X_encoded[var + f'_{target}_max_enc'] = X_encoded[var].map(target_max)

    X_encoded.drop('price', axis=1, inplace=True)
    return X_encoded

def target_encode_evaluation_data_with_multiple_targets(X_eval, X_train, y_train):
    # Copy the evaluation data for target encoding
    X_encoded_eval = X_eval.copy()

    # Add 'price' to training data for target encoding calculations
    X_train_encoded = X_train.copy()
    X_train_encoded['price'] = y_train

    # Apply target encoding using statistics from the TRAINING data
    for target in targets_for_encoding:
        for var in categorical_vars:
            # Calculate statistics for target encoding using TRAINING data
            target_mean = X_train_encoded.groupby(var)[target].mean()
            target_max = X_train_encoded.groupby(var)[target].max()

            # Apply the calculated statistics to the EVALUATION data
            X_encoded_eval[var + f'_{target}_mean_enc'] = X_encoded_eval[var].map(target_mean)
            X_encoded_eval[var + f'_{target}_max_enc'] = X_encoded_eval[var].map(target_max)

    # Handle any NaN values which might occur if there's a new category in evaluation data
    X_encoded_eval.fillna(X_train_encoded.mean(), inplace=True)
    return X_encoded_eval


# Apply target encoding to X using statistics from itself
X_encoded_from_X = target_encode_with_specified_targets(X, y, targets_for_price)

# Apply target encoding to X again using statistics from X_all
X_encoded_from_all = target_encode_evaluation_data_with_multiple_targets(X, X_all, y)

# Combine both encoded data for X
X_combined_encoded = pd.concat([X_encoded_from_X, X_encoded_from_all], axis=1)


X_new_test_encoded_from_X = target_encode_evaluation_data_with_multiple_targets(X_new_test, X, y)

# Apply target encoding to X_new_test using statistics from X_all and y for 'odometer' and 'year'
X_new_test_encoded_from_all = target_encode_evaluation_data_with_multiple_targets(X_new_test, X_all, y)

# Combine both encoded test data
X_new_test_combined_encoded = pd.concat([X_new_test_encoded_from_X, X_new_test_encoded_from_all], axis=1)

X_new_test_combined_encoded.head()


<ipython-input-47-f1f7075dd6f6>:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_encoded[var + f'_{target}_mean_enc'] = X_encoded[var].map(target_mean)
<ipython-input-47-f1f7075dd6f6>:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_encoded[var + f'_{target}_max_enc'] = X_encoded[var].map(target_max)
<ipython-input-47-f1f7075dd6f6>:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using 

,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,...,size_paint_color_price_mean_enc,size_paint_color_price_max_enc,size_state_price_mean_enc,size_state_price_max_enc,type_paint_color_price_mean_enc,type_paint_color_price_max_enc,type_state_price_mean_enc,type_state_price_max_enc,paint_color_state_price_mean_enc,paint_color_state_price_max_enc
id,,,,,,,,,,,,,,,,,,,,,
27532,357,2015,42,0,3,2,92553,0,0,1,...,9.226448,11.480599,9.330922,11.400977,9.059561,11.480599,9.248722,11.146849,9.119626,11.096652
27533,271,2013,63,3,3,2,134385,6,0,1,...,9.274698,11.196967,9.120814,10.853909,9.056116,11.131680,8.937455,10.853909,9.296213,11.267306
27534,2,2011,74,2,3,2,102489,0,0,1,...,9.427975,11.400977,9.185139,11.273221,9.056116,11.131680,8.890560,10.624906,9.152396,11.273221
27535,74,2016,52,0,5,0,64310,0,0,0,...,8.980289,10.955899,9.150509,10.821237,9.059561,11.480599,9.268791,10.883992,9.083048,10.550119
27536,134,1999,48,0,6,2,180839,5,0,0,...,9.030135,11.095818,9.097225,11.087421,9.096323,11.095818,9.243937,11.087421,8.912066,10.834194


In [ ]:
X_combined_encoded

,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,...,transmission_size_price_mean_enc,transmission_size_price_max_enc,transmission_type_price_mean_enc,transmission_type_price_max_enc,drive_size_price_mean_enc,drive_size_price_max_enc,drive_type_price_mean_enc,drive_type_price_max_enc,size_type_price_mean_enc,size_type_price_max_enc
id,,,,,,,,,,,,,,,,,,,,,
0,215,1949,39,0,5,2,115148,0,1,2,...,9.002582,11.169801,9.210240,10.781287,9.436102,11.302353,9.366915,10.843475,9.363252,10.818157
1,321,2013,72,1,6,2,172038,0,0,2,...,9.359863,11.480599,9.007278,11.201074,9.373652,11.318369,9.264435,11.201074,8.955201,11.201074
2,358,1998,46,2,5,2,152492,0,0,1,...,9.359863,11.480599,9.309844,11.480599,8.939408,11.176921,9.048696,10.969353,9.290272,11.480599
3,3,2014,46,0,3,2,104118,0,1,1,...,9.002582,11.169801,9.049841,11.273221,9.015053,11.169801,9.048696,10.969353,9.262217,11.302353
4,264,2005,46,0,5,2,144554,0,1,1,...,9.002582,11.169801,8.793447,11.146027,9.015053,11.169801,8.917410,10.942385,9.042572,11.131680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27527,360,2008,46,2,5,2,26660,0,0,2,...,8.919402,10.969353,9.678240,11.465100,9.207698,10.887979,9.542093,11.290582,9.078606,10.380125
27528,337,2007,46,0,6,2,108072,0,0,2,...,9.359863,11.480599,9.618981,11.400977,9.373652,11.318369,9.442330,11.318369,9.581019,11.400977
27529,269,2019,52,3,5,2,139908,0,0,0,...,9.196960,11.302353,9.309844,11.480599,9.464871,11.280665,9.442506,11.480599,9.262217,11.302353


In [ ]:
X_new_test_combined_encoded

,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,...,transmission_size_price_mean_enc,transmission_size_price_max_enc,transmission_type_price_mean_enc,transmission_type_price_max_enc,drive_size_price_mean_enc,drive_size_price_max_enc,drive_type_price_mean_enc,drive_type_price_max_enc,size_type_price_mean_enc,size_type_price_max_enc
id,,,,,,,,,,,,,,,,,,,,,
27532,357,2015,42,0,3,2,92553,0,0,1,...,9.359863,11.480599,9.309844,11.480599,8.939408,11.176921,9.048696,10.969353,9.290272,11.480599
27533,271,2013,63,3,3,2,134385,6,0,1,...,9.196960,11.302353,9.007278,11.201074,9.015053,11.169801,8.917410,10.942385,9.042572,11.131680
27534,2,2011,74,2,3,2,102489,0,0,1,...,9.359863,11.480599,9.007278,11.201074,8.939408,11.176921,8.917410,10.942385,8.955201,11.201074
27535,74,2016,52,0,5,0,64310,0,0,0,...,9.196960,11.302353,9.309844,11.480599,9.464871,11.280665,9.442506,11.480599,9.262217,11.302353
27536,134,1999,48,0,6,2,180839,5,0,0,...,9.196960,11.302353,9.309844,11.480599,9.464871,11.280665,9.442506,11.480599,9.262217,11.302353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55064,121,2016,47,0,5,2,90902,5,0,1,...,9.359863,11.480599,9.678240,11.465100,8.939408,11.176921,9.060004,10.849240,9.638499,11.465100
55065,174,2012,63,0,3,2,27234,5,0,1,...,9.196960,11.302353,9.007278,11.201074,9.015053,11.169801,8.917410,10.942385,9.042572,11.131680
55066,251,2002,39,0,5,2,99761,0,0,2,...,9.359863,11.480599,9.207098,11.196967,9.373652,11.318369,9.273507,11.196967,9.101938,11.156222


In [ ]:
# nan_train = X_train_scaled.isna().sum()
# nan_test = X_test_scaled.isna().sum()

# print("NaN values in X_train_scaled:")
# print(nan_train[nan_train > 0])  # Only columns with NaN values

# print("\nNaN values in X_test_scaled:")
# print(nan_test[nan_test > 0])  # Only columns with NaN values


In [ ]:
X_train_scaled, X_val_scaled, y_train, y_val_scaled= train_test_split(X_combined_encoded , y, test_size=0.3, random_state=42)
X_test_scaled=X_new_test_combined_encoded

In [ ]:
X_val_scaled

,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,...,transmission_size_price_mean_enc,transmission_size_price_max_enc,transmission_type_price_mean_enc,transmission_type_price_max_enc,drive_size_price_mean_enc,drive_size_price_max_enc,drive_type_price_mean_enc,drive_type_price_max_enc,size_type_price_mean_enc,size_type_price_max_enc
id,,,,,,,,,,,,,,,,,,,,,
21617,367,2006,48,0,3,2,156081,0,1,1,...,8.769920,10.942385,8.793447,11.146027,8.831379,10.969353,8.917410,10.942385,8.838854,10.942385
21326,165,2008,58,1,3,2,143323,0,1,1,...,8.769920,10.942385,8.793447,11.146027,8.831379,10.969353,8.917410,10.942385,8.838854,10.942385
5317,120,2016,68,0,6,0,130992,0,0,0,...,9.359863,11.480599,9.678240,11.465100,9.605630,11.480599,9.769985,11.465100,9.638499,11.465100
19378,46,2005,48,1,3,2,131332,1,1,1,...,8.769920,10.942385,8.879510,11.107660,8.831379,10.969353,8.899706,10.793660,8.872018,10.747745
2540,184,2000,39,0,3,2,126589,0,1,2,...,9.002582,11.169801,8.793447,11.146027,9.436102,11.302353,9.264435,11.201074,9.042572,11.131680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4373,220,2014,74,3,3,2,123231,5,0,1,...,9.196960,11.302353,9.007278,11.201074,9.015053,11.169801,8.917410,10.942385,9.042572,11.131680
19809,178,1997,9,1,6,2,45072,0,1,2,...,9.183054,11.308616,9.484746,11.290856,9.373652,11.318369,9.542093,11.290582,9.638499,11.465100
5728,366,2007,46,0,5,2,252286,0,0,1,...,9.359863,11.480599,9.007278,11.201074,8.939408,11.176921,8.917410,10.942385,8.955201,11.201074


In [ ]:
X_test_scaled

,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,...,transmission_size_price_mean_enc,transmission_size_price_max_enc,transmission_type_price_mean_enc,transmission_type_price_max_enc,drive_size_price_mean_enc,drive_size_price_max_enc,drive_type_price_mean_enc,drive_type_price_max_enc,size_type_price_mean_enc,size_type_price_max_enc
id,,,,,,,,,,,,,,,,,,,,,
27532,357,2015,42,0,3,2,92553,0,0,1,...,9.359863,11.480599,9.309844,11.480599,8.939408,11.176921,9.048696,10.969353,9.290272,11.480599
27533,271,2013,63,3,3,2,134385,6,0,1,...,9.196960,11.302353,9.007278,11.201074,9.015053,11.169801,8.917410,10.942385,9.042572,11.131680
27534,2,2011,74,2,3,2,102489,0,0,1,...,9.359863,11.480599,9.007278,11.201074,8.939408,11.176921,8.917410,10.942385,8.955201,11.201074
27535,74,2016,52,0,5,0,64310,0,0,0,...,9.196960,11.302353,9.309844,11.480599,9.464871,11.280665,9.442506,11.480599,9.262217,11.302353
27536,134,1999,48,0,6,2,180839,5,0,0,...,9.196960,11.302353,9.309844,11.480599,9.464871,11.280665,9.442506,11.480599,9.262217,11.302353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55064,121,2016,47,0,5,2,90902,5,0,1,...,9.359863,11.480599,9.678240,11.465100,8.939408,11.176921,9.060004,10.849240,9.638499,11.465100
55065,174,2012,63,0,3,2,27234,5,0,1,...,9.196960,11.302353,9.007278,11.201074,9.015053,11.169801,8.917410,10.942385,9.042572,11.131680
55066,251,2002,39,0,5,2,99761,0,0,2,...,9.359863,11.480599,9.207098,11.196967,9.373652,11.318369,9.273507,11.196967,9.101938,11.156222


In [ ]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.model_selection import KFold

# LightGBMモデルの初期化
all_predictions = []
params = {'objective': 'regression',  # 最小化させるべき損失関数
          "max_depth":128,
         "learing_rate":0.05,
         'random_state': 42,  # 乱数シード
         'boosting_type': 'dart',  # boosting_typegbdt
         #'n_estimators': 200,
          'early_stopping_rounds':50,
         'n_estimators':10000# 最大学習サイクル数。early_stopping使用時は大きな値を入力
         }

model = lgb.LGBMRegressor(**params)
def mape(y_true, y_pred):
    """Calculate Mean Absolute Percentage Error"""
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def evaluate_with_cv_and_evalset(model, X, y,X_test2,y_test2,X_new_test,all_predictions, n_splits=20):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    mape_scores = []


    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # 訓練データの一部を評価データセットとして分割
        X_train_part, X_eval, y_train_part, y_eval = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

        # eval_setとして評価データセットを指定
        eval_set = [(X_eval, y_eval)]

        model.fit(X_train_part, y_train_part,  eval_metric='rmse',
                  eval_set=eval_set)

        # 予測とMAPEスコアの計算
        y_pred = model.predict(X_test2)
        # 予測を実行
        fold_predictions = model.predict(X_new_test)
        all_predictions.append(np.expm1(fold_predictions))

        current_mape = mape(np.expm1(y_test2), np.expm1(y_pred))
        mape_scores.append(current_mape)

    return np.mean(mape_scores),all_predictions
def remove_duplicate_columns(df):
    return df.loc[:,~df.columns.duplicated()]

X_train_scaled = remove_duplicate_columns(X_train_scaled)
X_val_scaled = remove_duplicate_columns(X_val_scaled)  # There seems to be a typo here as well. Should it be X_val_scaled?
X_test_scaled = remove_duplicate_columns(X_test_scaled)

# クロスバリデーションとeval_setを使用してモデルを評価
average_mape,all_predictions = evaluate_with_cv_and_evalset(model,X_train_scaled,y_train, X_val_scaled,y_val_scaled,X_test_scaled,all_predictions)
print(f"Average MAPE with Cross-Validation and eval_set: {average_mape:.2f}%")



[LightGBM] [Warning] Unknown parameter: learing_rate
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: learing_rate
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.038214 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15784
[LightGBM] [Info] Number of data points in the train set: 14646, number of used features: 177
[LightGBM] [Warning] Unknown parameter: learing_rate
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning

/usr/local/lib/python3.10/dist-packages/lightgbm/callback.py:288: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


[LightGBM] [Warning] Unknown parameter: learing_rate
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: learing_rate
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: learing_rate
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: learing_rate
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.037091 seconds.
You can set `force_col_wise

/usr/local/lib/python3.10/dist-packages/lightgbm/callback.py:288: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


[LightGBM] [Warning] Unknown parameter: learing_rate
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: learing_rate
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: learing_rate
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: learing_rate
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.040212 seconds.
You can set `force_col_wise

...

[LightGBM] [Warning] Unknown parameter: learing_rate
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: learing_rate
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: learing_rate
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: learing_rate
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.044738 seconds.
You can set `force_col_wise

/usr/local/lib/python3.10/dist-packages/lightgbm/callback.py:288: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


[LightGBM] [Warning] Unknown parameter: learing_rate
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: learing_rate
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: learing_rate
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: learing_rate
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.038155 seconds.
You can set `force_col_wise

/usr/local/lib/python3.10/dist-packages/lightgbm/callback.py:288: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


[LightGBM] [Warning] Unknown parameter: learing_rate
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: learing_rate
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Average MAPE with Cross-Validation and eval_set: 52.59%


In [ ]:
# 1. 平均予測値を計算
ensemble_predictions = np.mean(all_predictions, axis=0)

# 2. 平均予測値とテストデータのIDを結合して新しいデータフレームを作成
results_df = pd.DataFrame({
    'ID': X_test_scaled.index,
    'Predicted_Price': ensemble_predictions
})

# 3. 結果のデータフレームを表示
print(results_df.head())
results_df.to_csv("resukt.csv",encoding="cp932",index=False,header=False)

      ID  Predicted_Price
0  27532     12795.143108
1  27533      9206.576411
2  27534      6235.230280
3  27535     17362.122679
4  27536      4562.916921


In [ ]:
 results_df

,ID,Predicted_Price
0,27532,11185.862149
1,27533,4447.019365
2,27534,6012.076998
3,27535,17837.774542
4,27536,15062.671702
...,...,...
27532,55064,3630.960145
27533,55065,8455.373528
27534,55066,9195.930210
27535,55067,10122.250228


# 新しいセクション

In [ ]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

# Initialize models
models = [
    ("Linear Regression", LinearRegression()),
    ("Ridge Regression", Ridge()),
    ("Random Forest", RandomForestRegressor(random_state=42)),
    ("Gradient Boosting", GradientBoostingRegressor(random_state=42)),
    ("Support Vector Machine", SVR()),
    ("Neural Network", MLPRegressor(random_state=42)),
    ("XGBoost", xgb.XGBRegressor(random_state=42))
]

def mape(y_true, y_pred):
    """Calculate Mean Absolute Percentage Error"""
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


def evaluate_each_model_no_cv(models, X_train_scaled, X_test_scaled, y_train, y_test):
    model_mape_scores = {}
    #y_train_log = np.log1p(y_train)  # Log transformation of the training target
    for name, model in models:
        # Train the model with log-transformed target
        model.fit(X_train_scaled, y_train)

        # Predict and transform predictions back to original scale
        y_pred_log = model.predict(X_test_scaled)
        y_pred = np.expm1(y_pred_log)  # Convert from log scale to original scale
        y_test_exm = np.expm1(y_test)

        # Calculate MAPE with original scale
        current_mape = mape(y_test_exm, y_pred)
        model_mape_scores[name] = current_mape
    return model_mape_scores



def ensemble_and_evaluate_no_cv(models, weights, X_train_scaled, X_test_scaled, y_train, y_test):
    assert len(models) == len(weights), "Number of models must match number of weights."

    weighted_predictions = []
    for idx, (name, model) in enumerate(models):
        # Make predictions on the test set (assuming models are already trained)
        y_pred_log = model.predict(X_test_scaled)
        y_pred = np.expm1(y_pred_log)  # Convert from log scale to original scale
        weighted_predictions.append(y_pred * weights[idx])

    # Ensemble predictions (taking weighted sum of all model predictions)
    ensemble_pred = np.sum(weighted_predictions, axis=0)
    y_test_exm = np.expm1(y_test)
    ensemble_mape = mape(y_test_exm, ensemble_pred)

    return ensemble_mape



# Evaluate each model without cross-validation
average_mape_scores = evaluate_each_model_no_cv(models, X_train_scaled, X_test_scaled, y_train, y_test)
for model_name, mape_score in average_mape_scores.items():
    print(f"MAPE for {model_name}: {mape_score}")

# Ensemble evaluation without cross-validation using equal weights
weights_equal = [1/len(models) for _ in models]
mape_ensemble_equal = ensemble_and_evaluate_no_cv(models, weights_equal, X_train_scaled, X_test_scaled, y_train, y_test)
print("\nMAPE for Ensemble with equal weights:", mape_ensemble_equal)

# Ensemble evaluation without cross-validation using custom weights
weights_custom = [0.1, 0.1, 0.2, 0.3, 0.1, 0.2, 0.1]  # Added an additional weight for XGBoost
mape_ensemble_custom = ensemble_and_evaluate_no_cv(models, weights_custom, X_train_scaled, X_test_scaled, y_train, y_test)
print("\nMAPE for Ensemble with custom weights:", mape_ensemble_custom)


# ... The rest of the code for ensemble remains similar but ensure to handle log transformation


KeyboardInterrupt: ignored

In [ ]:

# 2. Predict using trained models
ensemble_predictions_log = np.zeros(len(X_new_test_scaled))  # Initialize with zeros
for idx, (name, model) in enumerate(models):
    y_pred_log = model.predict(X_new_test_scaled)  # Get predictions in log scale
    ensemble_predictions_log += weights_equal[idx] * y_pred_log  # Add weighted predictions

ensemble_predictions = np.expm1(ensemble_predictions_log)  # Convert from log scale to original scale

# 3. Combine predicted values with the original test data IDs
results_df = pd.DataFrame({
    'ID': X_new_test.index,
    'Predicted_Price': ensemble_predictions
})

# 4. Display the resulting dataframe
print(results_df)


ValueError: ignored

In [ ]:

import pandas as pd
results_df.to_csv("resukt.csv",encoding="cp932",index=False,header=False)
